In [1]:
import librosa
import os
import os.path
import matplotlib.pyplot as plt
import matplotlib
import numpy as np
from pydub import AudioSegment

## Chuyển các file từ m4a thành wav

In [2]:
path = "D:/songs/"

os.chdir(path)

audio_files = os.listdir()

for file in audio_files:
    name, ext = os.path.splitext(file)
    if ext == ".m4a":
        m4a_sound = AudioSegment.from_file(file, "m4a")
        m4a_sound.export("D:/songs_wav/{0}.wav".format(name), format="wav") #đổi đường ổ tương nhé các gs

## MusicSearch

In [6]:
plt.rcParams['figure.figsize'] = (15, 5)

In [7]:
training_dir = 'D:/songs_wav/'
training_files = [os.path.join(training_dir, f) for f in os.listdir(training_dir)]

In [8]:
print(training_files)

['D:/songs_wav/10000HoursCover_-JungkookBTS_.wav', 'D:/songs_wav/22Break-OhWonder.wav', 'D:/songs_wav/24_5-MimiWebb.wav', 'D:/songs_wav/2step-EdSheeran.wav', 'D:/songs_wav/2TimesDimitriVegasEdit_-DinoWarri.wav', 'D:/songs_wav/2TimesDimitriVegasExtendedEdit_-D.wav', 'D:/songs_wav/3107-W_n_Duongg_Nau.wav', 'D:/songs_wav/3Times-ShyGhy_Tiny.wav', 'D:/songs_wav/3TimesExtendedMix_-ShyGhy_Tiny.wav', 'D:/songs_wav/3_WelcomeToTheShow-BRay.wav', 'D:/songs_wav/5minutesavectoi-Amir.wav', 'D:/songs_wav/99Luftballons-Kaleida.wav', 'D:/songs_wav/A-M-E-R-I-C-A-MotionlessInWhite.wav', 'D:/songs_wav/Abandon-LastDaysOfEden.wav', 'D:/songs_wav/Abracadaba-QveenHerby.wav', 'D:/songs_wav/Abracadabra-LastDaysOfEden.wav', 'D:/songs_wav/ACulDeGreve-Hanternoz.wav', 'D:/songs_wav/AdieuMonHomme-Pomme.wav', 'D:/songs_wav/Adrenalin-Power5.wav', 'D:/songs_wav/ADyingWish-Thulcandra.wav', 'D:/songs_wav/AFleurDeToi-Vitaa_Slimane.wav', 'D:/songs_wav/Afterlife-AdInfinitum.wav', 'D:/songs_wav/AfterYou-Gryffin_JasonRoss_Cal

In [9]:
def hash_func(vecs, projections):
    bools = np.dot(vecs, projections.T) > 0
    return [bool2int(bool_vec) for bool_vec in bools]

In [10]:
def bool2int(x):
    y = 0
    for i,j in enumerate(x):
        if j: y += 1<<i
    return y

In [11]:
bool2int([False, True, False, True])

10

In [12]:
X = np.random.randn(10,100)
P = np.random.randn(3,100)
hash_func(X, P)

[7, 4, 1, 6, 6, 2, 3, 0, 4, 6]

In [13]:
class Table:
    def __init__(self, hash_size, dim):
        self.table = dict()
        self.hash_size = hash_size
        self.projections =  np.random.randn(self.hash_size, dim)

    def add(self, vecs, label):
        entry = {'label': label}
        hashes = hash_func(vecs, self.projections)
        for h in hashes:
            if self.table.__contains__(h):
                self.table[h].append(entry)
            else:
                self.table[h] = [entry]

    def query(self, vecs):
        hashes = hash_func(vecs, self.projections)
        results = list()
        for h in hashes:
            if self.table.__contains__(h):
                results.extend(self.table[h])
        return results

In [14]:
class LSH:
    def __init__(self, dim):
        self.num_tables = 4
        self.hash_size = 8
        self.tables = list()
        for i in range(self.num_tables):
            self.tables.append(Table(self.hash_size, dim))
    
    def add(self, vecs, label):
        for table in self.tables:
            table.add(vecs, label)
    
    def query(self, vecs):
        results = list()
        for table in self.tables:
            results.extend(table.query(vecs))
        return results

    def describe(self):
        for table in self.tables:
            print(table.table)

In [15]:
class MusicSearch:
    
    def __init__(self, training_files):
        self.frame_size = 4096
        self.hop_size = 4000
        self.fv_size = 12
        self.lsh = LSH(self.fv_size)
        self.training_files = training_files
        self.num_features_in_file = dict()
        for f in self.training_files:
            self.num_features_in_file[f] = 0
                
    def train(self):
        for filepath in self.training_files:
            x, fs = librosa.load(filepath)
            features = librosa.feature.chroma_stft(x, fs, n_fft=self.frame_size, hop_length=self.hop_size).T
            self.lsh.add(features, filepath)
            self.num_features_in_file[filepath] += len(features)
                
    def query(self, filepath):
        x, fs = librosa.load(filepath)
        features = librosa.feature.chroma_stft(x, fs, n_fft=self.frame_size, hop_length=self.hop_size).T
        results = self.lsh.query(features)
        print('num results '+ str(len(results)))

        counts = dict()
        for r in results:
            if counts.__contains__(r['label']):
                counts[r['label']] += 1
            else:
                counts[r['label']] = 1
        for k in counts:
            counts[k] = float(counts[k])/self.num_features_in_file[k]
        return counts

In [ ]:
ms = MusicSearch(training_files)
ms.train()

In [ ]:
test_file = 'D:/testing.wav'
results = ms.query(test_file)

In [ ]:
for r in sorted(results, key=results.get, reverse=True):
    print(r, results[r])

### Testing: 
Bad Habits - Ed Sheraan.wav 